In [1]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Cargar y Preprocesamiento de los Datos

- Definición de Categorías

In [2]:
categorias = {
    "Reglamentos Generales y Código de Conducta": 0,
    "Reglamentos Académicos y de Evaluación": 1,
    "Reglamentos de Becas y Excelencia Académica": 2,
    "Reglamentos de Investigación": 3,
    "Reglamentos de Instalaciones y Uso de Recursos": 4
}

- Inicialización y Configuración de Directorio

In [4]:
import os
import pandas as pd

# Directorio donde están tus archivos de texto
data_dir = "/content/archivos"

# Inicializa listas para almacenar textos y etiquetas
texts = []
labels = []

- Mapeo de Archivos a Categorías

Define un diccionario que asocia cada archivo con su categoría correspondiente. Esto permite determinar la categoría del contenido sin necesidad de abrir el archivo para analizarlo.

In [5]:
# Define la lista de archivos y sus categorías respectivas
archivo_categoria_map = {
    "CodigoDeConductaSCETIC": "Reglamentos Generales y Código de Conducta",
    "ConjuntoDeReglamentosFACULTADDECIENCIASYTECNOLOGIA": "Reglamentos Generales y Código de Conducta",
    "CONVOCATORIADEBECASGESTION2024COMISIONDEBIENESTAR": "Reglamentos de Becas y Excelencia Académica",
    "ESTATUTOORGANICOUSFX2010": "Reglamentos Generales y Código de Conducta",
    "MODELOACADEMICO2010": "Reglamentos Académicos y de Evaluación",
    "plandeestudioscarrerassiscicdadtic": "Reglamentos Académicos y de Evaluación",
    "Politicasacademicas": "Reglamentos Académicos y de Evaluación",
    "REGDELACARRERADELDOCENTEINVESTIGADORUSFX": "Reglamentos Académicos y de Evaluación",
    "REGDELSISTEMADEINVESTIGACIONUSFX": "Reglamentos de Investigación",
    "Reglamentodecontroldelacontaminacionacusticaenlosprediosdelafacultaddetecnologia": "Reglamentos de Instalaciones y Uso de Recursos",
    "Reglamentodeviajesdeestudios": "Reglamentos Académicos y de Evaluación",
    "REGLAMENTODELREGIMENESTUDIANTIL": "Reglamentos Académicos y de Evaluación",
    "ReglamentoEspecificodeSancionesDisciplinarias": "Reglamentos Generales y Código de Conducta",
    "REGLAMENTOEXCELENCIAACADEMICA(BECAESTUDIO)": "Reglamentos de Becas y Excelencia Académica",
    "REGLAMENTOGENERALGRADUACIONTECNOLOGIA2015VERSIONFINAL1": "Reglamentos Académicos y de Evaluación",
    "REGLAMENTOINTERNO": "Reglamentos Generales y Código de Conducta",
    "Reglamentoparaelusoyfuncionamientodelainstalacionesdeportivasdelafacultad": "Reglamentos de Instalaciones y Uso de Recursos",
    "reglamentolabfisica": "Reglamentos de Instalaciones y Uso de Recursos",
    "reglamentodelclaustrouniversitariofiltradoSandraVillca": "Reglamentos Generales y Código de Conducta",
    "REGLAMENTOESPECIFICOSISTEMAORGANIZACIONADMINISTRATIVA": "Reglamentos Generales y Código de Conducta",
    "REGLAMENTOESPECIFICOSISTEMAPROGRAMACIONOPERACIONES2006": "Reglamentos Generales y Código de Conducta",
    "REGLAMENTOGENERALDEINTERACCIONSOCIAL": "Reglamentos Generales y Código de Conducta",
    "ReglamentoPgaengTecnologiaUsfxFinal": "Reglamentos Generales y Código de Conducta",
    "ReglamentoUsoRecursosIDH": "Reglamentos de Instalaciones y Uso de Recursos"
}

# Leer cada archivo y asignarle su categoría numérica
for archivo, categoria in archivo_categoria_map.items():
    try:
        # Intentar leer con UTF-8
        with open(os.path.join(data_dir, archivo + ".txt"), 'r', encoding='utf-8') as f:
            texts.append(f.read())
            labels.append(categorias[categoria])  # Asigna la categoría numérica
    except UnicodeDecodeError:
        # Si ocurre un error de decodificación, intentar con ISO-8859-1
        with open(os.path.join(data_dir, archivo + ".txt"), 'r', encoding='ISO-8859-1') as f:
            texts.append(f.read())
            labels.append(categorias[categoria])  # Asigna la categoría numérica

# Crear el DataFrame final
df = pd.DataFrame({'text': texts, 'label': labels})

# Visualizar las primeras filas
df.head()


,text,label
0,Código de conducta SCETIC\nIntroducción:\nLa i...,0
1,FACULTAD DE CIENCIAS Y TECNOLOGÍA\n\nING. MIGU...,0
2,BIENESTAR SO...,2
3,UNIVERSIDAD MAYOR REAL Y PONTIFICIA\nDE SAN FR...,0
4,"Universidad Mayor, Real y Pontificia de San\nF...",1


In [44]:
#df

Convertir los Datos en un Dataset Compatible

In [6]:
from datasets import Dataset

# Convertir el DataFrame en un Dataset
dataset = Dataset.from_pandas(df)

Tokenizar el Texto con un Modelo Preentrenado
- Seleccionaremos distilbert-base-uncased para esta tarea. Cargamos el tokenizador y aplicamos tokenización a los textos.

In [7]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenizar el texto
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

# Tokenizar todo el dataset
tokenized_dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Dividir el Conjunto de Datos

In [8]:
#Divide el conjunto de datos en entrenamiento y prueba para evaluar el rendimiento.
split_dataset = tokenized_dataset.train_test_split(test_size=0.2) # 80% para train y 20% para test
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [9]:
# Selecciona un ejemplo de `train_dataset`
example = train_dataset[0]

# Imprime el texto original
print("Texto original:")
print(example['text'])

# Imprime la representación numérica (tokens)
print("\nRepresentación numérica (tokens):")
print(example['input_ids'])


Texto original:
﻿UNIVERSIDAD MAYOR, REAL Y PONTIFICIA  DE SAN FRANCISCO XAVIER DE CHUQUISACA 
  

REGLAMENTO ESPECÍFICO DE SANCIONES  DISCIPLINARIAS 
(SECTOR ADMINISTRATIVO) 
Reglamento aprobado mediante Resolución Rectoral 007/2003 de 20/01/03 Homolagada por Resolución del H.C.U. 005/2003 de 14/05/03 
SUCRE - BOLIVIA
CONTENIDO 
REGLAMENTO ESPECÍFICO 
DE SANCIONES DISCIPLINARIAS 
TÍTULO I 
Disposiciones Generales 1 CAPÍTULO I 
Aspectos Generales del Reglamento 1 CAPÍTULO II 
De las Faltas Disciplinarias y sus Sanciones 3 CAPÍTULO III 
Del Control de Asistencia 7
Reglamento Específico de Sanciones Disciplinarias  
_______________________________________________________________________________ 
REGLAMENTO ESPECÍFICO DE SANCIONES DISCIPLINARIAS PARA EL  SECTOR ADMINISTRATIVO 
TITULO I 
DISPOSICIONES GENERALES  
CAPITULO I 
ASPECTOS GENERALES DEL REGLAMENTO 
Artículo 1. CONCEPTO Y CARACTERISTICAS GENERALES 
El Reglamento Específico de sanciones disciplinarias, es un conjunto ordenado de  p

**Cargar el Modelo Preentrenado y Configurar las Métricas**

In [10]:
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score

# Configurar el modelo
num_labels = len(categorias)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Definir las métricas
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Configurar el Entrenamiento**
 - **Configura los parámetros de entrenamiento**

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    weight_decay=0.01,
    report_to="none"  # Esto desactiva wandb
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**Entrenar el Modelo**

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Entrena el modelo
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.473159,0.400000,0.228571
2,No log,1.412814,0.400000,0.228571
3,No log,1.317944,0.400000,0.228571
4,No log,1.272700,0.600000,0.466667
5,No log,1.237671,0.600000,0.466667
6,No log,1.224173,0.600000,0.520000
7,No log,1.188257,0.600000,0.520000
8,No log,1.161767,0.600000,0.520000
9,No log,1.307224,0.600000,0.520000
10,No log,1.364254,0.600000,0.520000


TrainOutput(global_step=300, training_loss=0.13289269765218098, metrics={'train_runtime': 7010.7241, 'train_samples_per_second': 0.271, 'train_steps_per_second': 0.043, 'total_flos': 251701522944000.0, 'train_loss': 0.13289269765218098, 'epoch': 100.0})

**Evaluar el Modelo**
 - **Al finalizar el entrenamiento, evalúa el modelo para ver su rendimiento en el conjunto de prueba.**

In [13]:
trainer.evaluate()

{'eval_loss': 2.4912097454071045,
 'eval_accuracy': 0.6,
 'eval_f1': 0.52,
 'eval_runtime': 5.7057,
 'eval_samples_per_second': 0.876,
 'eval_steps_per_second': 0.175,
 'epoch': 100.0}

- eval_loss (Pérdida de Evaluación): una pérdida de aproximadamente 2.49 sugiere que el modelo no está funcionando de manera óptima, y es posible que necesite más entrenamiento o ajustes de hiperparámetros.

- eval_accuracy (Precisión de Evaluación): es del 60%, lo que significa que el modelo predice correctamente el 60% de las veces en el conjunto de prueba. Aunque es un rendimiento decente, podría mejorarse.

- eval_f1 (Puntaje F1 de Evaluación): El puntaje F1 ponderado es de 0.52. Este valor combina la precisión y la recuperación, dando un balance entre las dos. Un valor de F1 bajo sugiere que el modelo podría estar teniendo problemas con algunas clases, especialmente si hay desbalance entre ellas.


**Guardar el Modelo**
- guarda el modelo y el tokenizador para usarlos

In [14]:
model.save_pretrained("mi_modelo_clasificacion_universidad")
tokenizer.save_pretrained("mi_modelo_clasificacion_universidad")

('mi_modelo_clasificacion_universidad/tokenizer_config.json',
 'mi_modelo_clasificacion_universidad/special_tokens_map.json',
 'mi_modelo_clasificacion_universidad/vocab.txt',
 'mi_modelo_clasificacion_universidad/added_tokens.json',
 'mi_modelo_clasificacion_universidad/tokenizer.json')

# **Probar el Modelo con Nuevos Textos**
 - **realizar predicciones con textos nuevos, carga el modelo y tokenizador, y haz predicciones**

In [15]:
# Cargar modelo y tokenizador
from transformers import pipeline

clasificador = pipeline("text-classification", model="mi_modelo_clasificacion_universidad")

# Clasificar un nuevo texto
# Texto sacado de CodigoDeConductaSCETIC.txt que label0= Reglamentos Generales y Código de Conducta
nuevo_texto = "El comité de ética, será formado a la cabeza del director de ética y directores voluntarios para revisar la situación. Para las faltas leves y regulares, la responsabilidad y decisión será del DIRECTOR DE ÉTICA siempre y cuando escuche y revise la recomendación de los directores. Si la situación amerita una falta grave, el comité de ética, será formado a la cabeza del director de ética con 3 directores y  3 socios escogidos aleatoriamente que no sean afines con el involucrado para la revisión y una votación para la decisión de la sanción. Si el caso amerita una mayor investigación, existirá la posibilidad de ampliar el número del comité de ética, siempre y cuando sea en imparidad para evitar un empate"
prediccion = clasificador(nuevo_texto)
print(prediccion)


[{'label': 'LABEL_0', 'score': 0.9986103773117065}]


In [22]:
# Cargar modelo y tokenizador
from transformers import pipeline

clasificador = pipeline("text-classification", model="mi_modelo_clasificacion_universidad")

# Clasificar un nuevo texto
#Texto sacado de Politicasacademicas.txt que es label1=Reglamentos Académicos y de Evaluación
nuevo_texto = "destinarán los recursos recibidos por IDH, para los sectores de educación, salud y caminos, desarrollo productivo y todo lo que contribuya a la generación de fuentes de trabajo. Ley Nº 070, de 20 de diciembre de 2010, Ley de Educación “Avelino Siñani - Elizardo Pérez”, Artículo 56 dispone que las Universidades Públicas Autónomas se regirán por lo establecido en la Constitución Política del Estado."
prediccion = clasificador(nuevo_texto)
print(prediccion)

[{'label': 'LABEL_1', 'score': 0.9575468301773071}]


In [43]:
# Cargar modelo y tokenizador
from transformers import pipeline

clasificador = pipeline("text-classification", model="mi_modelo_clasificacion_universidad")

# Clasificar un nuevo texto
#Texto sacado de Reglamentoparaelusoyfuncionamientodelainstalacionesdeportivasdelafacultad.txt que es label4=Reglamentos de Instalaciones y Uso de Recursos
nuevo_texto = "El Encargado de Administración, luego de verificar si no se perjudica en el horario solicitado ninguna actividad programada, aceptará o rechazará la solicitud, indicando monto a ser cancelado por el solicitante, de acuerdo al tarifario aprobado y anexo en el presente reglamento, en Administración de la Facultad de Tecnología."
prediccion = clasificador(nuevo_texto)
print(prediccion)

[{'label': 'LABEL_0', 'score': 0.9980402588844299}]
